### Deep Learning for Images with PyTorch

#### Images
Images consist of pixels, each of which contains color information (ints from 0 - 255; one channel if grayscale and three if RGB).

#### Loading images to PyTorch
Use `ImageFolder` from `torchvision` to create a `Dataset` of images.

In [ ]:
import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms

train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(45),

    transforms.ToTensor(), # parse to tensoe
    transforms.Resize((128, 128)) # resize to 128x128px
])

# dataset_train = ImageFolder("data/clouds_train", transform=train_transforms)
# dataloader_train = DataLoader(dataset_train, batch_size=1, shuffle=True)
# image, label = next(iter(data_loader_train))

# image = image.squeeze().permute(1, 2, 0)
# print(image.shape)

# image, label = private

# @@@ Displaying an Image, @@@
import matplotlib.pyplot as plt
plt.imshow(image)
plt.show()